In [13]:
import os
import sys
import csv
import json

from numpy.linalg import svd
from math import sqrt
import numpy as np

# from procrustes import procrustes

In [25]:
def procrustes(X, Y, scaling=True, reflection='best'):
    """
    A port of MATLAB's `procrustes` function to Numpy.

    Procrustes analysis determines a linear transformation (translation,
    reflection, orthogonal rotation and scaling) of the points in Y to best
    conform them to the points in matrix X, using the sum of squared errors
    as the goodness of fit criterion.

        d, Z, [tform] = procrustes(X, Y)

    Inputs:
    ------------
    X, Y    
        matrices of target and input coordinates. they must have equal
        numbers of  points (rows), but Y may have fewer dimensions
        (columns) than X.

    scaling 
        if False, the scaling component of the transformation is forced
        to 1

    reflection
        if 'best' (default), the transformation solution may or may not
        include a reflection component, depending on which fits the data
        best. setting reflection to True or False forces a solution with
        reflection or no reflection respectively.

    Outputs
    ------------
    d       
        the residual sum of squared errors, normalized according to a
        measure of the scale of X, ((X - X.mean(0))**2).sum()

    Z
        the matrix of transformed Y-values

    tform   
        a dict specifying the rotation, translation and scaling that
        maps X --> Y

    """

    n,m = X.shape
    ny,my = Y.shape

    muX = X.mean(0)
    muY = Y.mean(0)

    X0 = X - muX
    Y0 = Y - muY

    ssX = (X0**2.).sum()
    ssY = (Y0**2.).sum()

    # centred Frobenius norm
    normX = np.sqrt(ssX)
    normY = np.sqrt(ssY)

    # scale to equal (unit) norm
    X0 /= normX
    Y0 /= normY

    if my < m:
        Y0 = np.concatenate((Y0, np.zeros(n, m-my)),0)

    # optimum rotation matrix of Y
    A = np.dot(X0.T, Y0)
    U,s,Vt = np.linalg.svd(A,full_matrices=False)
    V = Vt.T
    T = np.dot(V, U.T)

    if reflection is not 'best':

        # does the current solution use a reflection?
        have_reflection = np.linalg.det(T) < 0

        # if that's not what was specified, force another reflection
        if reflection != have_reflection:
            V[:,-1] *= -1
            s[-1] *= -1
            T = np.dot(V, U.T)

    traceTA = s.sum()

    if scaling:

        # optimum scaling of Y
        b = traceTA * normX / normY

        # standarised distance between X and b*Y*T + c
        d = 1 - traceTA**2

        # transformed coords
        Z = normX*traceTA*np.dot(Y0, T) + muX

    else:
        b = 1
        d = 1 + ssY/ssX - 2 * traceTA * normY / normX
        Z = normY*np.dot(Y0, T) + muX

    # transformation matrix
    if my < m:
        T = T[:my,:]
    c = muX - b*np.dot(muY, T)

    #transformation values 
    tform = {'rotation':T, 'scale':b, 'translation':c}

    return d, Z, tform

In [26]:
def flatten_input(image):
  processed = np.empty((0,51))
  for frame in image:
    intermediate = frame.flatten()
    processed = np.append(processed,[intermediate],axis=0)
  return processed

In [27]:
datasets_folder = 'datasets/train'
datasets_folder_path = os.path.abspath("datasets/train")

In [28]:
image_class = []
for class_folder in os.listdir(datasets_folder):
    for filename in os.listdir(datasets_folder_path+'/'+class_folder):
        image_path = 'datasets/train/'+class_folder+'/'+filename
        image_class.append({"image":image_path,"class":class_folder})

In [29]:
csv_columns = ['image','class']
dict_data = image_class

In [30]:
csv_file = 'dystonia_dataset_train.csv'
try:
    with open(csv_file,'w') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=csv_columns)
        writer.writeheader()
        for data in dict_data:
            writer.writerow(data)
except IOError:
    print('I/O error')

In [31]:
def data_normalization(data):
    normalized_data = np.empty((0,17,3))
    reference_frame = data[0]

    for frame in data:
        d, Z, tform = procrustes(reference_frame, frame)
        normalized_data = np.append(normalized_data,[Z],axis=0)
#         print(normalized_data)
    return normalized_data

In [32]:

def default(obj):
    if type(obj).__module__ == np.__name__:
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        else:
            return obj.item()
    raise TypeError('Unknown type:', type(obj))

In [33]:
image_class = []
for class_folder in os.listdir(datasets_folder):
    for filename in os.listdir(datasets_folder_path+'/'+class_folder):
        image_path = datasets_folder_path+'/'+class_folder+'/'+filename
        data  = np.load(image_path)
        subsampled_data = data[::3]
        normalized_data = data_normalization(subsampled_data)
        flattened_data = flatten_input(normalized_data)
        flattened_data = np.expand_dims(flattened_data, axis=2)
        serialized_data = json.dumps(flattened_data, default=default)
        image_class.append({"image":serialized_data,"class":class_folder})

In [34]:
csv_columns = ['image','class']
dict_data = image_class

In [35]:
csv_file = 'dystonia_dataset_train_normalized.csv'
try:
    with open(csv_file,'w') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=csv_columns)
        writer.writeheader()
        for data in dict_data:
            writer.writerow(data)
except IOError:
    print('I/O error')